# LangChain Simple Chat Model (Azure)

In [1]:
# !pip install langchain
# !pip install langchain-openai
# !pip install openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from agentquest.benchmarks.mastermind import MasterMindDriver, MasterMindUtils

game = MasterMindUtils.load_data(data_path="__default__", category="4 digits")[0]
game

'5918'

Initialize the prompt and the LLM

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Initialize the LLM
llm = AzureChatOpenAI(
    azure_deployment="gpt-4-default", api_version="2024-05-01-preview"
)

/tmp/ipykernel_243859/2839392786.py:6: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


Initialize the mastermind driver and get the first observation

In [6]:
driver = MasterMindDriver(game)
obs = driver.reset()
print(f"OBSERVATION: {obs.output}")

# Initalize the system prompt
chat_template = ChatPromptTemplate.from_messages([])
messages = chat_template.format_messages()

OBSERVATION: You are tasked to play the Mastermind game.
The host chooses a number and gives you the amount of digits. You have to guess the correct number as fast as you can.
The number can contain repetitions and any possible digit between: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.
At each round, you provide a number as a guess. At each step, the host provides you this information:
1. The number of correct digits in the wrong position.
2. The number of correct digits in the correct position.
The game ends when the host outputs 'You Won!'
Carefully choose your strategy. Avoid brute force.
The guess must be in the following format:
Guess: <number>
Start guessing the 4 digits number.


Start the LLM loop solving the task

In [20]:
from langchain.schema.messages import HumanMessage, AIMessage

step_cnt = 0
while not obs.success and obs.can_proceed and step_cnt < 20:
    # Provide the observation to the LLM
    messages.append(HumanMessage(content=obs.output))

    # Get the LLM output
    agent_output = llm(messages).content
    print(agent_output)

    # Provide the choice to the LLM as history
    messages.append(AIMessage(content=agent_output))

    obs = driver.step_raw(raw_text=agent_output)
    print(f"OBSERVATION: {obs.output}")

    step_cnt += 1

Guess: 5917
OBSERVATION: Wrong! Your guess has 3 correct digits in the correct positions and 0 correct digit in the wrong position. Keep guessing.
Guess: 5918
OBSERVATION: You Won!


In [21]:
driver.metrics.export(
    repetition_function_kwargs={"theta_a": 1, "num_execution_steps": 20}
)

{'goal': '5918',
 'success': True,
 'actions': [{'value': '1234'},
  {'value': '2345'},
  {'value': '3456'},
  {'value': '4567'},
  {'value': '5678'},
  {'value': '5670'},
  {'value': '5679'},
  {'value': '5698'},
  {'value': '5869'},
  {'value': '5986'},
  {'value': '5896'},
  {'value': '9685'},
  {'value': '8569'},
  {'value': '6598'},
  {'value': '6958'},
  {'value': '6985'},
  {'value': '8956'},
  {'value': '8596'},
  {'value': '5968'},
  {'value': '5969'},
  {'value': '5960'},
  {'value': '5961'},
  {'value': '5619'},
  {'value': '5916'},
  {'value': '5917'},
  {'value': '5918'}],
 'states': [{'value': '1234'},
  {'value': '2345'},
  {'value': '3456'},
  {'value': '4567'},
  {'value': '5678'},
  {'value': '5670'},
  {'value': '5679'},
  {'value': '5698'},
  {'value': '5869'},
  {'value': '5986'},
  {'value': '5896'},
  {'value': '9685'},
  {'value': '8569'},
  {'value': '6598'},
  {'value': '6958'},
  {'value': '6985'},
  {'value': '8956'},
  {'value': '8596'},
  {'value': '5968'}